In [2]:
import numpy as np
import pandas as pd
from pupil_functions import *

# tracking delay
## TCB_GOOD

In [3]:
col = ["block", "trial", "task", "sent", "received"]
timesTCB = pd.DataFrame(columns = col)

In [4]:
dfTCB = load_manual_annotations("/Users/selena_zhang/Desktop/000")

In [5]:
#change none's to nearest numbers
for idx, num in dfTCB.iloc[:, 4].items():
    if num == None:
        point = idx
        while point >= 0:
            #finds the nearest filled cell that is before it
            point -= 1
            val = dfTCB.iloc[point, 4]
            if val is not None:
                dfTCB.set_value(idx, "trial", val)
                break
#first two must be manually done
dfTCB.set_value(0, "trial", 1)
dfTCB.set_value(1, "trial", 1);

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [7]:
dfTCB[5:15]

,block,exp_event,task,timestamp,trial
5,1,Image onset for trial 1 in block 1,Image onset,1.591225e+09,1
6,None,image onset,,1.591225e+09,1
7,None,testing_image_onset,,1.591225e+09,1
8,None,adjust,,1.591225e+09,1
9,1,Adjust Task begins for trial 1 in block 1,Adjust task,1.591225e+09,1
10,1,Adjust Task ends for trial 1 in block 1,,1.591225e+09,1
11,1,Ending trial 1 in block 1,End trial,1.591225e+09,1
12,1,Begin for trial 2 in block 1,Trial begins,1.591225e+09,2
13,1,Fixation dot begins for trial 2 in block 1,Fixation dot,1.591225e+09,2
14,None,fixation,,1.591225e+09,2


In [8]:
for idx, event in dfTCB.iterrows():
    #first 10 trials
    if float(dfTCB.iloc[idx, 4]) > 10:
        break
    #fixation
    if dfTCB.iloc[idx, 2] == "Fixation dot":
        task = "Fixation dot"
        trial = dfTCB.iloc[idx, 4]
        block = dfTCB.iloc[idx, 0]
        sent = dfTCB.iloc[idx, 3]
        if dfTCB.iloc[idx + 1, 1] == "fixation":
            received = dfTCB.iloc[idx + 1, 3]
        else:
            received = dfTCB.iloc[idx - 1, 3]
        row = [block, trial, task, sent, received]
        timesTCB.loc[len(timesTCB), :] = row
    #image onset
    elif dfTCB.iloc[idx, 2] == "Image onset":
        task = "Image onset"
        trial = dfTCB.iloc[idx, 4]
        block = dfTCB.iloc[idx, 0]
        sent = dfTCB.iloc[idx, 3]
        if dfTCB.iloc[idx + 1, 1] == "image onset":
            received = dfTCB.iloc[idx + 1, 3]
        else:
            received = dfTCB.iloc[idx - 1, 3]        
        row = [block, trial, task, sent, received]
        timesTCB.loc[len(timesTCB), :] = row
    elif dfTCB.iloc[idx, 2] == "Adjust task":
        task = "Adjust task"
        trial = dfTCB.iloc[idx, 4]
        block = dfTCB.iloc[idx, 0]
        sent = dfTCB.iloc[idx, 3]
        if dfTCB.iloc[idx + 1, 1] == "adjust":
            received = dfTCB.iloc[idx + 1, 3]
        else:
            received = dfTCB.iloc[idx - 1, 3]
        row = [block, trial, task, sent, received]
        timesTCB.loc[len(timesTCB), :] = row
    else:
        pass

In [9]:
timesTCB["delay"] = timesTCB["sent"] - timesTCB["received"]

In [10]:
avg = timesTCB["delay"].mean()

In [11]:
if avg < 0:
    timesTCB["fixedlag"] = timesTCB["received"] + avg #avg was negative
elif avg > 0:
    timesTCB["fixedlag"] = timesTCB["received"] - avg

In [12]:
timesTCB

,block,trial,task,sent,received,delay,fixedlag
0,1,1,Fixation dot,1.59123e+09,1.59123e+09,-0.0300457,1.59123e+09
1,1,1,Image onset,1.59123e+09,1.59123e+09,-0.139507,1.59123e+09
2,1,1,Adjust task,1.59123e+09,1.59123e+09,0.00205851,1.59123e+09
3,1,2,Fixation dot,1.59123e+09,1.59123e+09,-0.0401266,1.59123e+09
4,1,2,Image onset,1.59123e+09,1.59123e+09,-0.0865505,1.59123e+09
5,1,2,Adjust task,1.59123e+09,1.59123e+09,-0.00968146,1.59123e+09
6,1,3,Fixation dot,1.59123e+09,1.59123e+09,-0.0155714,1.59123e+09
7,1,3,Image onset,1.59123e+09,1.59123e+09,-0.0523214,1.59123e+09
8,1,3,Adjust task,1.59123e+09,1.59123e+09,-0.0153713,1.59123e+09
9,1,4,Fixation dot,1.59123e+09,1.59123e+09,-0.0435736,1.59123e+09
